**Описание задания:**

1.Подготовить мини-корпус (4-5 текстов или до 10 тысяч токенов) с разметкой ключевых слов.
Желательно указать источник корпуса и описать, в каком виде там были представлены ключевые слова.

2.Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.

3.Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25).

4.Оценить точность, полноту, F-меру выбранных методов относительно имеющейся разметки.

5.Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.



**1. Подготовка корпуса**

Тексты 4 статей взяты с сайта http://sci-article.ru/gryps.php?i=lingvistika.
Ключевые слова в данных статьях указаны отдельно после аннотаций. Я скопировала их как строки, а потом создала массив массивов слов.

In [1]:
clu_words = ['эмоционально-оценочные коммуникемы; национально-культурная специфика; коммуникативное поведение; восприятие речи',\
             'лексика со значением «медицинский работник»; тематическая группа; исторический период; английский язык',\
             'информационно-коммуникационные технологии; социальные сети; дидактические свойства; английский язык; обучение иностранным языкам; изучение иностранных языков',\
             'звукоподражания; крики птиц; ассоциативные связи']

In [2]:
clues_orig = []
for i in clu_words:
    new = i.split('; ')
    clues_orig.append(new)

In [3]:
for i in clues_orig:
    print(i)

['эмоционально-оценочные коммуникемы', 'национально-культурная специфика', 'коммуникативное поведение', 'восприятие речи']
['лексика со значением «медицинский работник»', 'тематическая группа', 'исторический период', 'английский язык']
['информационно-коммуникационные технологии', 'социальные сети', 'дидактические свойства', 'английский язык', 'обучение иностранным языкам', 'изучение иностранных языков']
['звукоподражания', 'крики птиц', 'ассоциативные связи']


Далее создаю массив текстов.

In [4]:
texts = []
for i in range(1,5):
    name = f'text_{i}.txt'
    file = open(name, 'r', encoding = 'utf-8')
    text = file.read()
    texts.append(text)
    file.close()

In [5]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [6]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [7]:
def normalize(text):
    tokens = tokenizer.tokenize(text)
    lemmas = [m.parse(t)[0].normal_form for t in tokens] #0 - первый разбор
    return lemmas

Нормализуем ключевые слова. Это понадобится позже при оценивании метрик, чтобы такие примеры, как 'социальная сеть' и 'социальные сети', считались один и тем же ключевым словом, а не разными.

In [8]:
clus_t = []
for i in clues_orig:
    clues = []
    for j in i:
        clu_tok = ' '.join(normalize(j))
        clues.append(clu_tok)
    clus_t.append(clues)     

In [9]:
clus_t

[['эмоционально оценочный коммуникем',
  'национальный культурный специфика',
  'коммуникативный поведение',
  'восприятие речь'],
 ['лексика с значение медицинский работник',
  'тематический группа',
  'исторический период',
  'английский язык'],
 ['информационно коммуникационный технология',
  'социальный сеть',
  'дидактический свойство',
  'английский язык',
  'обучение иностранный язык',
  'изучение иностранный язык'],
 ['звукоподражание', 'крик птица', 'ассоциативный связь']]

**2. Самостоятельная разметка ключевых слов**

Я выделила следующие ключевые слова:

In [10]:
my_clu_words_1 = ['эмоционально-оценочные коммуникемы',\
               'коммуникативное поведение']

In [11]:
my_clu_words_2 = ['тематическая группа "медицинский работник"',\
               'английский словарь']

In [12]:
my_clu_words_3 = ['социальная сеть','обучение английскому языку',\
               'дидактические свойства']

In [13]:
my_clu_words_4 = ['звукоподражательные глаголы','ассоциативные связи']

In [14]:
my_clues = []
my_clues.append(my_clu_words_1)
my_clues.append(my_clu_words_2)
my_clues.append(my_clu_words_3)
my_clues.append(my_clu_words_4)

Нормализуем и их

In [15]:
my_clus_t = []
for i in my_clues:
    clues = []
    for j in i:
        clu_tok = ' '.join(normalize(j))
        clues.append(clu_tok)
    my_clus_t.append(clues)    

In [16]:
my_clus_t[0]

['эмоционально оценочный коммуникем', 'коммуникативный поведение']

Среди различий можно отметить то, что у меня некоторые ключевые слова более узкие, чем в статьях. Например, "звукоподражательные глаголы", а не "звукоподражание", или "обучение английскому языку" вместо "обучение иностранным языкам". Также какая-то пара ключевых сочетаний может быть у меня объединена в одно, например, "тематическая группа «медицинский работник»" вместо "лексика со значением «медицинский работник»" и "тематическая группа". В остальном выделенные ключевые слова совпадают, за тем исключением, что я в принципе выделила меньше слов.

**3&4. Применение трёх автоматических методов и их оценка**

Попробуем теперь использовать автоматические методы извлечения ключевых слов.

1)RAKE

In [17]:
import RAKE

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Дарья\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [20]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for token in simple_word_tokenize(text):
        lemmas.append(
            m.parse(token)[0].normal_form
        )
    return ' '.join(lemmas)

In [21]:
results = []
for i in texts:
    result = rake.run(normalize_text(i), maxWords=3, minFrequency=2)
    results.append(result)

In [22]:
results[0][:10]

[('эмоционально-оценочный коммуникем', 10.11111111111111),
 ('эмоционально-оценочный коммуникть', 10.0),
 ('рассмотреть данный положение', 8.2),
 ('героиня произведение дж', 8.166666666666666),
 ('героиня произведение', 5.166666666666666),
 ('герой произведение', 4.833333333333334),
 ('said leclare', 4.333333333333334),
 ('kinsella « i', 4.222222222222222),
 ('русский язык', 4.166666666666666),
 ('во-первых', 4.0)]

In [23]:
test_res = []
for i in results:
    mas = []
    for j in i:
        mas.append(j[0])
    test_res.append(mas)

In [24]:
test_res[0][:10]

['эмоционально-оценочный коммуникем',
 'эмоционально-оценочный коммуникть',
 'рассмотреть данный положение',
 'героиня произведение дж',
 'героиня произведение',
 'герой произведение',
 'said leclare',
 'kinsella « i',
 'русский язык',
 'во-первых']

In [25]:
def metr_test(true, test):
    a = b = c = 0
    
    for i in test:
        if i in true:
            a += 1 #выданные релевантные результаты
        else:
            b += 1 #выданные нерелевантные результаты
            
    for i in true:
        if i not in test:
            c += 1 #невыданные релевантные результаты
            
    if (a+b) != 0:
        p = a/(a+b) #precision
    else:
        p = 0
        
    if (a+c) != 0:
        r = a/(a+c) #recall
    else:
        r = 0
    
    if (p+r) != 0:
        f1 = 2*(p*r)/(r+p) #f1-score
    else:
        f1 = 0
    
    return [p, r, f1]

Посчитаем точность, полноту и f1-score для оригинальных и полученных метрикой ключевых слов

In [26]:
for i in range(4):
    print(metr_test(clus_t[i], test_res[i]))

[0.0, 0.0, 0]
[0.034482758620689655, 0.5, 0.06451612903225806]
[0.125, 0.5, 0.2]
[0.03125, 0.6666666666666666, 0.05970149253731343]


Посчитаем точность, полноту и f1-score для выделенных мною и полученных метрикой ключевых слов

In [27]:
for i in range(4):
    print(metr_test(my_clus_t[i], test_res[i]))

[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.08333333333333333, 0.6666666666666666, 0.14814814814814814]
[0.015625, 0.5, 0.030303030303030304]


2)TextRank

In [28]:
from gensim.summarization import keywords as kw

In [29]:
results_2 = []
for i in texts:
    result = kw(normalize_text(i), pos_filter=[], scores=True)
    results_2.append(result)

Уберем из результатов стоп-слова

In [30]:
new_results_2 = []
for i in results_2:
    mas = []
    for j in i:
        if j[0] not in stop:
            mas.append(j)
    new_results_2.append(mas)

In [31]:
new_results_2[0][:10]

[('свои', 0.19724061799023707),
 ('эмоционально коммуникем', 0.16961226216496567),
 ('единица', 0.15464561293777837),
 ('являться', 0.12410964662018299),
 ('речевои', 0.1188198023462976),
 ('русскии язык', 0.11589643091473692),
 ('которыи', 0.11072540229112474),
 ('эмоциональныи', 0.10753684815200491),
 ('дать', 0.10624129530053349),
 ('ситуация', 0.10106087790200315)]

In [32]:
test_res_2 = []
for i in new_results_2:
    mas = []
    for j in i:
        mas.append(j[0])
    test_res_2.append(mas)

In [33]:
test_res_2[0][:10]

['свои',
 'эмоционально коммуникем',
 'единица',
 'являться',
 'речевои',
 'русскии язык',
 'которыи',
 'эмоциональныи',
 'дать',
 'ситуация']

Теперь оценим метрику, опять сравним оригинальные и мои ключевые слова с теми, что выдал TestRank

In [34]:
for i in range(4):
    print(metr_test(clus_t[i], test_res_2[i]))

[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.01639344262295082, 0.6666666666666666, 0.032]


In [35]:
for i in range(4):
    print(metr_test(my_clus_t[i], test_res_2[i]))

[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]


Видно, что результаты здесь гораздо хуже, чем у Rake, даже если убрать стоп-слова, которые присутствовали среди результатов изначально

3)Tf-idf

In [36]:
texts_tok = []
for i in texts:
    new_t = normalize(i)
    s = ' '.join(new_t)
    texts_tok.append(s)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [38]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(texts_tok)

In [39]:
all_words = np.array(tfidf_vectorizer.get_feature_names())
matrix = X.toarray()

In [40]:
idx = []
for row in matrix:
    print(row)
    idx_sort = np.argsort(list(row))
    idx.append(idx_sort[::-1][:10]) #будем брать первые десять слов с наибольшим tf-idf score
idx

[0.         0.01191928 0.         ... 0.         0.         0.        ]
[0.0388579 0.        0.        ... 0.        0.        0.       ]
[0. 0. 0. ... 0. 0. 0.]
[0.         0.         0.01329274 ... 0.02658548 0.01329274 0.01329274]


[array([1085, 2351, 1539, 2327, 1316, 1886,  865, 1906, 1312, 2371],
       dtype=int64),
 array([1164,  333, 1155, 1205, 1154,  137, 1204, 1157, 1646,  661],
       dtype=int64),
 array([1951, 2032, 1425, 1265, 2371, 1830,  504, 1639, 1002, 2327],
       dtype=int64),
 array([1798, 1125, 1112, 2371,  930,  965,  933,  710,  934,  244],
       dtype=int64)]

In [41]:
results_3 = []
for i in idx:
    res = []
    for j in i:
        res.append(all_words[j])
    results_3.append(res)

In [42]:
results_3[0]

['коммуникем',
 'эмоционально',
 'оценочный',
 'что',
 'не',
 'речь',
 'единица',
 'русский',
 'национальный',
 'язык']

In [43]:
for i in range(4):
    print(metr_test(clus_t[i], results_3[i]))

[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.1, 0.3333333333333333, 0.15384615384615383]


In [44]:
for i in range(4):
    print(metr_test(my_clus_t[i], results_3[i]))

[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]
[0.0, 0.0, 0]


Здесь результаты тоже плохие. В данном случае это связано с тем, что благодаря tf-idf матрице мы узнаем релевантные для текстов слова, но не их сочетания. Чтобы немного сгладить картину, можно разбить ключевые слова по токенам и посмотреть попадания чисто по ним.

In [45]:
all_text_words = []
for i in clus_t:
    text_words = []
    for j in i:
        words = j.split()
        for q in words:
            text_words.append(q)
    all_text_words.append(text_words)

In [46]:
all_text_words[0]

['эмоционально',
 'оценочный',
 'коммуникем',
 'национальный',
 'культурный',
 'специфика',
 'коммуникативный',
 'поведение',
 'восприятие',
 'речь']

In [47]:
all_my_text_words = []
for i in my_clus_t:
    text_words = []
    for j in i:
        words = j.split()
        for q in words:
            text_words.append(q)
    all_my_text_words.append(text_words)

In [48]:
all_my_text_words[0]

['эмоционально', 'оценочный', 'коммуникем', 'коммуникативный', 'поведение']

In [49]:
for i in range(4):
    print(metr_test(all_text_words[i], results_3[i]))

[0.5, 0.5, 0.5]
[0.1, 0.09090909090909091, 0.09523809523809525]
[0.5, 0.38461538461538464, 0.4347826086956522]
[0.3, 0.6, 0.4]


In [50]:
for i in range(4):
    print(metr_test(all_my_text_words[i], results_3[i]))

[0.3, 0.6, 0.4]
[0.1, 0.16666666666666666, 0.125]
[0.5, 0.7142857142857143, 0.588235294117647]
[0.1, 0.25, 0.14285714285714288]


Теперь стало лучше, хотя немного некорректно

**5.Описание проблем и их возможное решение**

Одна из проблем, которая ухудшила результаты всех метрик, это не самая корректная нормализация ключевых слов, которые были выделены в оригинальном тексте и которые я разметила вручную. Минус нормализации заключался в том, что из словосочетаний пропадали также и необходимые дефисы, которые однако оставались при токенизации у разных метрик, например, "эмоционально-оценочный коммуникем". Подобным образом вполне могли бы токенизироваться иначе и какие-нибудь другие необычные выражения. Чтобы решить эту проблему, стоит уделить чуть большее внимания функции токенизации, учтя в ней такие случаи.

Маленький минус TextRank заключался также в том, что он не учитывал стоп-слова. Но их можно убрать уже из самих результатов, как и было проделанно в этой работе.

Недочет метрики с tf-idf был уже описан выше. Я попробовала исправить его не самым изящным способом. Для более правильного результата, следовало бы, наоборот, не разбить ключевые слова на токены, а посчитать tf-idf score, например, для биграмм и триграмм.

В итоге, можно сказать, что TextRank справился хуже всех, Rake позакал неплохие результаты, а tf-idf был лучшим, хотя работал только со словами, а не словосочетаниями.